In [1]:
!pip install chromadb

  Obtaining dependency information for chromadb from https://files.pythonhosted.org/packages/e9/4c/91e52c4180cf6e6a8f348d8ef90783ab7139a772a8887e8c8bfcf8f886ce/chromadb-0.4.10-py3-none-any.whl.metadata
  Obtaining dependency information for pydantic<2.0,>=1.9 from https://files.pythonhosted.org/packages/f9/e7/8dc6b418c3eefd15274e93e58ebd7ed95a650cc0ee3f38673bb56a8aa3f7/pydantic-1.10.12-cp311-cp311-macosx_11_0_arm64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.3/149.3 kB 2.4 MB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for chroma-hnswlib==0.7.3 from https://files.pythonhosted.org/packages/11/7a/673ccb9bb2faf9cf655d9040e970c02a96645966e06837fde7d10edf242a/chroma_hnswlib-0.7.3-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for fastapi<0.100.0,>=0.95.2 from https://files.pythonhosted.org/packages/73/eb/03b691afa0b5ffa1e93ed34f97ec1e7855c758efbdcfb16c209af0b0506b/fastapi-0.99.1-py3-none-any.whl.metadata
  Obtaining dependency i

In [8]:
!pip install openai

  Obtaining dependency information for openai from https://files.pythonhosted.org/packages/ae/59/911d6e5f1d7514d79c527067643376cddcf4cb8d1728e599b3b03ab51c69/openai-0.28.0-py3-none-any.whl.metadata
  Obtaining dependency information for aiohttp from https://files.pythonhosted.org/packages/47/10/33abd984a476e314afdb4711fbd0aac1b25927676fa591445537da3aee98/aiohttp-3.8.5-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for async-timeout<5.0,>=4.0.0a3 from https://files.pythonhosted.org/packages/a7/fa/e01228c2938de91d47b307831c62ab9e4001e747789d0b05baf779a6488c/async_timeout-4.0.3-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 1.7 MB/s eta 0:00:00
  Obtaining dependency information for frozenlist>=1.1.1 from https://files.pythonhosted.org/packages/9d/8b/8ab8143541b2c5fff4189fad7853e61d30e4ec4749ebf91e1d598c4e7c57/frozenlist-1.4.0-cp311-cp311-macosx_11_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.

In [3]:
import subprocess
import json

buf = subprocess.check_output("tf blocks-json openai,passwords,api", shell=True, text=True)
key = json.loads(buf)[0]["lines"][-1]
print(len(key))
# print(key)

51


In [2]:
import chromadb

chroma_client = chromadb.PersistentClient(
    path="./vector-db",
)

OpenAI charges $0.0001/1k tokens. Not sure how many tokens there are in the things I'm submitting, but 1k isn't much in my world, so keep an eye on it. See #openai, #api for how to check usage.

Maybe try a cache made from an in-process KV store (like BoltDB, but for Python) if it gets wasteful.

In [4]:
from chromadb.utils import embedding_functions
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=key,
    model_name="text-embedding-ada-002"
)

In [5]:
collection_name = "freight_train_embeddings"

Run the first time with a fresh database:

In [8]:
collection = chroma_client.get_or_create_collection(
    name=collection_name,
    metadata={"hnsw:space": "cosine"}, # [l2, cosine, ip] are options for distance measure
    embedding_function=openai_ef
)

TODO: How to clear data out of the database without deleting the files? Read Chroma documentation. Matt says it's short.

To load data into the database:

In [10]:
buf = subprocess.check_output("tf blocks-json 'freight train'", shell=True, text=True)
blocks = json.loads(buf)

In [11]:
print(len(blocks))

434


This hits the rate limits hard if run on all the blocks. Need some way to slow it down. How depends on whether the embeddings API can take multiple items per request and whether Chroma will take advantage of that if multiple documents are added at once. Either way, have to bring it down to 1,000,000 TPM and 3,000 RPM.

In [24]:
import uuid

for block in blocks[:4]:
    for line in block["lines"]:
        collection.add(
            documents=[line],
            metadatas=[{
                "dateline": block["dateline"],
            }],
            ids=[str(uuid.uuid4())],
        )

In [9]:
print(collection.count())

22


To query the database:

In [27]:
results = collection.query(
    query_texts=[
        "freight train",
    ],
    n_results=12,
    include=[
        # "ids" is always included
        # "embeddings",
        "documents",
        "metadatas",
        "distances"
    ]
)

* embeddings: lots and lots of numbers
* documents: the stuff you supplied
* metadatas: the stuff you supplied
* distances: a single number per result. lower is better

In [28]:
print(results["documents"])

[["the need for this is what we're trying to prevent", "the need for this is what we're trying to prevent", 'then you can say that', 'then you can say that', 'building a practical computer program together', 'building a practical computer program together', 'leads to complexity.', 'leads to complexity.', 'even if they can do so easily.', 'sometimes, "i know what you meant"', 'sometimes, "i know what you meant"', 'you and the computer are intelligent, well meaning people']]


Garbage in, garbage out!

In [29]:
print(results["metadatas"])

[[{'foo': 'foo'}, {'dateline': '2022-11-16 12:18:12'}, {'dateline': '2022-11-17 09:48:30'}, {'foo': 'foo'}, {'foo': 'foo'}, {'dateline': '2022-11-17 08:48:09'}, {'dateline': '2022-12-22 12:12:41'}, {'foo': 'foo'}, {'dateline': '2022-12-22 12:12:41'}, {'foo': 'foo'}, {'dateline': '2022-12-22 12:12:41'}, {'foo': 'foo'}]]
